In [98]:
import panel as pn
import joblib
import numpy as np
from sklearn.preprocessing import StandardScaler

pn.extension()

# Load the saved model and scaler
try:
    loaded_model = joblib.load('random_forest_model.joblib')
    scaler = joblib.load('scaler.joblib')
    print("Model type:", type(loaded_model))
    print("Scaler type:", type(scaler))
except Exception as e:
    print(f"Error loading model or scaler: {e}")
    raise

# Updated disaster type mapping
disaster_type_mapping = {
    0: "Flood", 1: "Storm", 2: "Earthquake", 3: "Epidemic", 4: "Landslide",
    5: "Drought", 6: "Extreme temperature", 7: "Wildfire", 8: "Volcanic activity",
    9: "Insect infestation", 10: "Mass movement (dry)", 11: "Glacial lake outburst",
    12: "Fog", 13: "Impact", 14: "Animal accident"
}

# Min and max values for each feature
min_max_values = {
    'Year': (1900.0, 2021.0),
    'Dis Mag Scale': (0.0, 4.0),
    'Dis Mag Value': (-57.0, 13025874.0),
    'Longitude': (0.0, 2439.0),
    'Latitude': (0.0, 2372.0)
}

# Country mapping
index_to_country = {0: 'Afghanistan', 1: 'Albania', 2: 'Algeria', 3: 'American Samoa', 4: 'Angola', 5: 'Anguilla', 6: 'Antigua and Barbuda', 7: 'Argentina', 8: 'Armenia', 9: 'Australia', 10: 'Austria', 11: 'Azerbaijan', 12: 'Azores Islands', 13: 'Bahamas (the)', 14: 'Bahrain', 15: 'Bangladesh', 16: 'Barbados', 17: 'Belarus', 18: 'Belgium', 19: 'Belize', 20: 'Benin', 21: 'Bermuda', 22: 'Bhutan', 23: 'Bolivia (Plurinational State of)', 24: 'Bosnia and Herzegovina', 25: 'Botswana', 26: 'Brazil', 27: 'Brunei Darussalam', 28: 'Bulgaria', 29: 'Burkina Faso', 30: 'Burundi', 31: 'Cabo Verde', 32: 'Cambodia', 33: 'Cameroon', 34: 'Canada', 35: 'Canary Is', 36: 'Cayman Islands (the)', 37: 'Central African Republic', 38: 'Chad', 39: 'Chile', 40: 'China', 41: 'Colombia', 42: 'Comoros (the)', 43: 'Congo (the Democratic Republic of the)', 44: 'Congo (the)', 45: 'Cook Islands (the)', 46: 'Costa Rica', 47: 'Croatia', 48: 'Cuba', 49: 'Cyprus', 50: 'Czech Republic (the)', 51: 'Czechoslovakia', 52: "Côte d'Ivoire", 53: 'Denmark', 54: 'Djibouti', 55: 'Dominica', 56: 'Dominican Republic (the)', 57: 'Ecuador', 58: 'Egypt', 59: 'El Salvador', 60: 'Equatorial Guinea', 61: 'Eritrea', 62: 'Estonia', 63: 'Ethiopia', 64: 'Fiji', 65: 'Finland', 66: 'France', 67: 'French Guiana', 68: 'French Polynesia', 69: 'Gabon', 70: 'Gambia (the)', 71: 'Georgia', 72: 'Germany', 73: 'Germany Dem Rep', 74: 'Germany Fed Rep', 75: 'Ghana', 76: 'Greece', 77: 'Grenada', 78: 'Guadeloupe', 79: 'Guam', 80: 'Guatemala', 81: 'Guinea', 82: 'Guinea-Bissau', 83: 'Guyana', 84: 'Haiti', 85: 'Honduras', 86: 'Hong Kong', 87: 'Hungary', 88: 'Iceland', 89: 'India', 90: 'Indonesia', 91: 'Iran (Islamic Republic of)', 92: 'Iraq', 93: 'Ireland', 94: 'Isle of Man', 95: 'Israel', 96: 'Italy', 97: 'Jamaica', 98: 'Japan', 99: 'Jordan', 100: 'Kazakhstan', 101: 'Kenya', 102: 'Kiribati', 103: "Korea (the Democratic People's Republic of)", 104: 'Korea (the Republic of)', 105: 'Kuwait', 106: 'Kyrgyzstan', 107: "Lao People's Democratic Republic (the)", 108: 'Latvia', 109: 'Lebanon', 110: 'Lesotho', 111: 'Liberia', 112: 'Libya', 113: 'Lithuania', 114: 'Luxembourg', 115: 'Macao', 116: 'Macedonia (the former Yugoslav Republic of)', 117: 'Madagascar', 118: 'Malawi', 119: 'Malaysia', 120: 'Maldives', 121: 'Mali', 122: 'Marshall Islands (the)', 123: 'Martinique', 124: 'Mauritania', 125: 'Mauritius', 126: 'Mexico', 127: 'Micronesia (Federated States of)', 128: 'Moldova (the Republic of)', 129: 'Mongolia', 130: 'Montenegro', 131: 'Montserrat', 132: 'Morocco', 133: 'Mozambique', 134: 'Myanmar', 135: 'Namibia', 136: 'Nepal', 137: 'Netherlands (the)', 138: 'Netherlands Antilles', 139: 'New Caledonia', 140: 'New Zealand', 141: 'Nicaragua', 142: 'Niger (the)', 143: 'Nigeria', 144: 'Niue', 145: 'Northern Mariana Islands (the)', 146: 'Norway', 147: 'Oman', 148: 'Pakistan', 149: 'Palau', 150: 'Palestine, State of', 151: 'Panama', 152: 'Papua New Guinea', 153: 'Paraguay', 154: 'Peru', 155: 'Philippines (the)', 156: 'Poland', 157: 'Portugal', 158: 'Puerto Rico', 159: 'Qatar', 160: 'Romania', 161: 'Russian Federation (the)', 162: 'Rwanda', 163: 'Réunion', 164: 'Saint Barthélemy', 165: 'Saint Helena, Ascension and Tristan da Cunha', 166: 'Saint Kitts and Nevis', 167: 'Saint Lucia', 168: 'Saint Martin (French Part)', 169: 'Saint Vincent and the Grenadines', 170: 'Samoa', 171: 'Sao Tome and Principe', 172: 'Saudi Arabia', 173: 'Senegal', 174: 'Serbia', 175: 'Serbia Montenegro', 176: 'Seychelles', 177: 'Sierra Leone', 178: 'Singapore', 179: 'Sint Maarten (Dutch part)', 180: 'Slovakia', 181: 'Slovenia', 182: 'Solomon Islands', 183: 'Somalia', 184: 'South Africa', 185: 'South Sudan', 186: 'Soviet Union', 187: 'Spain', 188: 'Sri Lanka', 189: 'Sudan (the)', 190: 'Suriname', 191: 'Swaziland', 192: 'Sweden', 193: 'Switzerland', 194: 'Syrian Arab Republic', 195: 'Taiwan (Province of China)', 196: 'Tajikistan', 197: 'Tanzania, United Republic of', 198: 'Thailand', 199: 'Timor-Leste', 200: 'Togo', 201: 'Tokelau', 202: 'Tonga', 203: 'Trinidad and Tobago', 204: 'Tunisia', 205: 'Turkey', 206: 'Turkmenistan', 207: 'Turks and Caicos Islands (the)', 208: 'Tuvalu', 209: 'Uganda', 210: 'Ukraine', 211: 'United Arab Emirates (the)', 212: 'United Kingdom of Great Britain and Northern Ireland (the)', 213: 'United States of America (the)', 214: 'Uruguay', 215: 'Uzbekistan', 216: 'Vanuatu', 217: 'Venezuela (Bolivarian Republic of)', 218: 'Viet Nam', 219: 'Virgin Island (British)', 220: 'Virgin Island (U.S.)', 221: 'Wallis and Futuna', 222: 'Yemen', 223: 'Yemen Arab Rep', 224: 'Yemen P Dem Rep', 225: 'Yugoslavia', 226: 'Zambia', 227: 'Zimbabwe'}

# Create input widgets
year_input = pn.widgets.FloatInput(name='Year', value=2000, start=min_max_values['Year'][0], end=min_max_values['Year'][1])
dis_mag_scale_input = pn.widgets.FloatInput(name='Dis Mag Scale', value=0, start=min_max_values['Dis Mag Scale'][0], end=min_max_values['Dis Mag Scale'][1])
dis_mag_value_input = pn.widgets.FloatInput(name='Dis Mag Value', value=0, start=min_max_values['Dis Mag Value'][0], end=min_max_values['Dis Mag Value'][1])
country_input = pn.widgets.Select(name='Country', options=list(index_to_country.values()))
longitude_input = pn.widgets.FloatInput(name='Longitude', value=0, start=min_max_values['Longitude'][0], end=min_max_values['Longitude'][1])
latitude_input = pn.widgets.FloatInput(name='Latitude', value=0, start=min_max_values['Latitude'][0], end=min_max_values['Latitude'][1])

# Create output panes
prediction_output = pn.pane.Markdown('')
probabilities_output = pn.pane.Markdown('')
debug_output = pn.pane.Markdown('')

def predict(*events):
    debug_output.object = "Prediction started..."
    try:
        # Get input values
        year = year_input.value
        dis_mag_scale = dis_mag_scale_input.value
        dis_mag_value = dis_mag_value_input.value
        country = list(index_to_country.values()).index(country_input.value)
        longitude = longitude_input.value
        latitude = latitude_input.value
        
        debug_output.object += f"\nInput values: {year}, {dis_mag_scale}, {dis_mag_value}, {country_input.value} (index: {country}), {longitude}, {latitude}"
        
        # Create input array
        input_array = np.array([[year, dis_mag_scale, dis_mag_value, country, longitude, latitude]])
        debug_output.object += f"\nInput array shape: {input_array.shape}"
        
        # Scale input
        input_array_scaled = scaler.transform(input_array)
        debug_output.object += f"\nScaled input shape: {input_array_scaled.shape}"
        
        # Make prediction
        prediction = loaded_model.predict(input_array_scaled)
        debug_output.object += f"\nRaw prediction: {prediction}"
        
        probabilities = loaded_model.predict_proba(input_array_scaled)
        debug_output.object += f"\nProbabilities shape: {probabilities.shape}"
        
        predicted_type_int = int(prediction[0])
        predicted_type_label = disaster_type_mapping.get(predicted_type_int, "Unknown")
        
        prediction_output.object = f"Predicted Disaster Type: {predicted_type_label} (Class {predicted_type_int})"
        
        prob_str = "Probabilities:\n"
        for i, prob in enumerate(probabilities[0]):
            prob_str += f"{disaster_type_mapping.get(i, f'Type {i}')}: {prob:.2%}\n"
        probabilities_output.object = prob_str
        
        debug_output.object += "\nPrediction completed successfully"
    except Exception as e:
        debug_output.object += f"\nAn error occurred: {str(e)}"
        import traceback
        debug_output.object += f"\n{traceback.format_exc()}"

# Create a button to trigger prediction
predict_button = pn.widgets.Button(name='Predict', button_type='primary')
predict_button.on_click(predict)

# Create information about input ranges
input_info = pn.pane.Markdown("""
### Input Ranges:
- Year: 1900 to 2021
- Disaster Magnitude Scale: 0 to 4
- Disaster Magnitude Value: -57 to 13,025,874
- Country: Select from the dropdown
- Longitude: 0 to 2439
- Latitude: 0 to 2372

Please enter values within these ranges for accurate predictions.
""")

# Create the layout
app = pn.Column(
    pn.pane.Markdown("# Disaster Type Prediction"),
    input_info,
    pn.Row(year_input, dis_mag_scale_input, dis_mag_value_input),
    pn.Row(country_input, longitude_input, latitude_input),
    predict_button,
    prediction_output,
    probabilities_output,
    debug_output
)

# Display the app
app.servable()

Model type: <class 'sklearn.ensemble._forest.RandomForestClassifier'>
Scaler type: <class 'sklearn.preprocessing._data.StandardScaler'>


Column
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Row
        [0] FloatInput(end=2021.0, name='Year', start=1900.0, value=2000)
        [1] FloatInput(end=4.0, name='Dis Mag Scale', start=0.0)
        [2] FloatInput(end=13025874.0, name='Dis Mag Value', start=-57.0)
    [3] Row
        [0] Select(name='Country', options=['Afghanistan', ...], value='Afghanistan')
        [1] FloatInput(end=2439.0, name='Longitude', start=0.0)
        [2] FloatInput(end=2372.0, name='Latitude', start=0.0)
    [4] Button(button_type='primary', name='Predict')
    [5] Markdown(str)
    [6] Markdown(str)
    [7] Markdown(str)

In [109]:
import panel as pn
import param
import joblib
import numpy as np
from sklearn.preprocessing import StandardScaler

pn.extension(design='material')

# Load the saved model and scaler
try:
    loaded_model = joblib.load('random_forest_model.joblib')
    scaler = joblib.load('scaler.joblib')
    print("Model type:", type(loaded_model))
    print("Scaler type:", type(scaler))
except Exception as e:
    print(f"Error loading model or scaler: {e}")
    raise

# Updated disaster type mapping
disaster_type_mapping = {
    0: "Flood", 1: "Storm", 2: "Earthquake", 3: "Epidemic", 4: "Landslide",
    5: "Drought", 6: "Extreme temperature", 7: "Wildfire", 8: "Volcanic activity",
    9: "Insect infestation", 10: "Mass movement (dry)", 11: "Glacial lake outburst",
    12: "Fog", 13: "Impact", 14: "Animal accident"
}

# Country mapping
index_to_country = {0: 'Afghanistan', 1: 'Albania', 2: 'Algeria', 3: 'American Samoa', 4: 'Angola', 5: 'Anguilla', 6: 'Antigua and Barbuda', 7: 'Argentina', 8: 'Armenia', 9: 'Australia', 10: 'Austria', 11: 'Azerbaijan', 12: 'Azores Islands', 13: 'Bahamas (the)', 14: 'Bahrain', 15: 'Bangladesh', 16: 'Barbados', 17: 'Belarus', 18: 'Belgium', 19: 'Belize', 20: 'Benin', 21: 'Bermuda', 22: 'Bhutan', 23: 'Bolivia (Plurinational State of)', 24: 'Bosnia and Herzegovina', 25: 'Botswana', 26: 'Brazil', 27: 'Brunei Darussalam', 28: 'Bulgaria', 29: 'Burkina Faso', 30: 'Burundi', 31: 'Cabo Verde', 32: 'Cambodia', 33: 'Cameroon', 34: 'Canada', 35: 'Canary Is', 36: 'Cayman Islands (the)', 37: 'Central African Republic', 38: 'Chad', 39: 'Chile', 40: 'China', 41: 'Colombia', 42: 'Comoros (the)', 43: 'Congo (the Democratic Republic of the)', 44: 'Congo (the)', 45: 'Cook Islands (the)', 46: 'Costa Rica', 47: 'Croatia', 48: 'Cuba', 49: 'Cyprus', 50: 'Czech Republic (the)', 51: 'Czechoslovakia', 52: "Côte d'Ivoire", 53: 'Denmark', 54: 'Djibouti', 55: 'Dominica', 56: 'Dominican Republic (the)', 57: 'Ecuador', 58: 'Egypt', 59: 'El Salvador', 60: 'Equatorial Guinea', 61: 'Eritrea', 62: 'Estonia', 63: 'Ethiopia', 64: 'Fiji', 65: 'Finland', 66: 'France', 67: 'French Guiana', 68: 'French Polynesia', 69: 'Gabon', 70: 'Gambia (the)', 71: 'Georgia', 72: 'Germany', 73: 'Germany Dem Rep', 74: 'Germany Fed Rep', 75: 'Ghana', 76: 'Greece', 77: 'Grenada', 78: 'Guadeloupe', 79: 'Guam', 80: 'Guatemala', 81: 'Guinea', 82: 'Guinea-Bissau', 83: 'Guyana', 84: 'Haiti', 85: 'Honduras', 86: 'Hong Kong', 87: 'Hungary', 88: 'Iceland', 89: 'India', 90: 'Indonesia', 91: 'Iran (Islamic Republic of)', 92: 'Iraq', 93: 'Ireland', 94: 'Isle of Man', 95: 'Israel', 96: 'Italy', 97: 'Jamaica', 98: 'Japan', 99: 'Jordan', 100: 'Kazakhstan', 101: 'Kenya', 102: 'Kiribati', 103: "Korea (the Democratic People's Republic of)", 104: 'Korea (the Republic of)', 105: 'Kuwait', 106: 'Kyrgyzstan', 107: "Lao People's Democratic Republic (the)", 108: 'Latvia', 109: 'Lebanon', 110: 'Lesotho', 111: 'Liberia', 112: 'Libya', 113: 'Lithuania', 114: 'Luxembourg', 115: 'Macao', 116: 'Macedonia (the former Yugoslav Republic of)', 117: 'Madagascar', 118: 'Malawi', 119: 'Malaysia', 120: 'Maldives', 121: 'Mali', 122: 'Marshall Islands (the)', 123: 'Martinique', 124: 'Mauritania', 125: 'Mauritius', 126: 'Mexico', 127: 'Micronesia (Federated States of)', 128: 'Moldova (the Republic of)', 129: 'Mongolia', 130: 'Montenegro', 131: 'Montserrat', 132: 'Morocco', 133: 'Mozambique', 134: 'Myanmar', 135: 'Namibia', 136: 'Nepal', 137: 'Netherlands (the)', 138: 'Netherlands Antilles', 139: 'New Caledonia', 140: 'New Zealand', 141: 'Nicaragua', 142: 'Niger (the)', 143: 'Nigeria', 144: 'Niue', 145: 'Northern Mariana Islands (the)', 146: 'Norway', 147: 'Oman', 148: 'Pakistan', 149: 'Palau', 150: 'Palestine, State of', 151: 'Panama', 152: 'Papua New Guinea', 153: 'Paraguay', 154: 'Peru', 155: 'Philippines (the)', 156: 'Poland', 157: 'Portugal', 158: 'Puerto Rico', 159: 'Qatar', 160: 'Romania', 161: 'Russian Federation (the)', 162: 'Rwanda', 163: 'Réunion', 164: 'Saint Barthélemy', 165: 'Saint Helena, Ascension and Tristan da Cunha', 166: 'Saint Kitts and Nevis', 167: 'Saint Lucia', 168: 'Saint Martin (French Part)', 169: 'Saint Vincent and the Grenadines', 170: 'Samoa', 171: 'Sao Tome and Principe', 172: 'Saudi Arabia', 173: 'Senegal', 174: 'Serbia', 175: 'Serbia Montenegro', 176: 'Seychelles', 177: 'Sierra Leone', 178: 'Singapore', 179: 'Sint Maarten (Dutch part)', 180: 'Slovakia', 181: 'Slovenia', 182: 'Solomon Islands', 183: 'Somalia', 184: 'South Africa', 185: 'South Sudan', 186: 'Soviet Union', 187: 'Spain', 188: 'Sri Lanka', 189: 'Sudan (the)', 190: 'Suriname', 191: 'Swaziland', 192: 'Sweden', 193: 'Switzerland', 194: 'Syrian Arab Republic', 195: 'Taiwan (Province of China)', 196: 'Tajikistan', 197: 'Tanzania, United Republic of', 198: 'Thailand', 199: 'Timor-Leste', 200: 'Togo', 201: 'Tokelau', 202: 'Tonga', 203: 'Trinidad and Tobago', 204: 'Tunisia', 205: 'Turkey', 206: 'Turkmenistan', 207: 'Turks and Caicos Islands (the)', 208: 'Tuvalu', 209: 'Uganda', 210: 'Ukraine', 211: 'United Arab Emirates (the)', 212: 'United Kingdom of Great Britain and Northern Ireland (the)', 213: 'United States of America (the)', 214: 'Uruguay', 215: 'Uzbekistan', 216: 'Vanuatu', 217: 'Venezuela (Bolivarian Republic of)', 218: 'Viet Nam', 219: 'Virgin Island (British)', 220: 'Virgin Island (U.S.)', 221: 'Wallis and Futuna', 222: 'Yemen', 223: 'Yemen Arab Rep', 224: 'Yemen P Dem Rep', 225: 'Yugoslavia', 226: 'Zambia', 227: 'Zimbabwe'}

class DisasterPredictor(param.Parameterized):
    year = param.Selector(default=2000, objects=list(range(1900, 2101)))
    dis_mag_scale = param.Selector(default=0, objects=list(range(5)))
    dis_mag_value = param.Selector(default=0, objects=[-57, 0, 100, 1000, 10000, 100000, 1000000, 13025874])
    country = param.ObjectSelector(default='Afghanistan', objects=list(index_to_country.values()))
    longitude = param.Selector(default=0, objects=list(range(0, 2440, 10)))
    latitude = param.Selector(default=0, objects=list(range(0, 2373, 10)))
    
    prediction = param.String(default='')
    probabilities = param.String(default='')
    
    def predict(self, event):
        try:
            country_index = list(index_to_country.values()).index(self.country)
            input_array = np.array([[self.year, self.dis_mag_scale, self.dis_mag_value, 
                                     country_index, self.longitude, self.latitude]])
            input_array_scaled = scaler.transform(input_array)
            prediction = loaded_model.predict(input_array_scaled)
            probabilities = loaded_model.predict_proba(input_array_scaled)
            
            predicted_type_int = int(prediction[0])
            predicted_type_label = disaster_type_mapping.get(predicted_type_int, "Unknown")
            
            self.prediction = f"Predicted Disaster Type: {predicted_type_label} (Class {predicted_type_int})"
            
            prob_str = "Probabilities:\n"
            for i, prob in enumerate(probabilities[0]):
                prob_str += f"{disaster_type_mapping.get(i, f'Type {i}')}: {prob:.2%}\n"
            self.probabilities = prob_str
            
        except Exception as e:
            self.prediction = f"An error occurred: {str(e)}"
            self.probabilities = ''

predictor = DisasterPredictor()

year_widget = pn.widgets.Select.from_param(predictor.param.year, name="Year")
dis_mag_scale_widget = pn.widgets.Select.from_param(predictor.param.dis_mag_scale, name="Disaster Magnitude Scale")
dis_mag_value_widget = pn.widgets.Select.from_param(predictor.param.dis_mag_value, name="Disaster Magnitude Value")
country_widget = pn.widgets.Select.from_param(predictor.param.country, name="Country")
longitude_widget = pn.widgets.Select.from_param(predictor.param.longitude, name="Longitude")
latitude_widget = pn.widgets.Select.from_param(predictor.param.latitude, name="Latitude")

predict_button = pn.widgets.Button(name='Predict', button_type='primary')
predict_button.on_click(predictor.predict)

input_panel = pn.Column(
    pn.pane.Markdown("## Input Parameters"),
    year_widget,
    dis_mag_scale_widget,
    dis_mag_value_widget,
    country_widget,
    longitude_widget,
    latitude_widget,
    predict_button
)

output_panel = pn.Column(
    pn.pane.Markdown("## Prediction Results"),
    pn.bind(lambda x: x, predictor.param.prediction),
    pn.bind(lambda x: x, predictor.param.probabilities)
)

# Create information about input ranges
input_info = pn.pane.Markdown("""
### Input Ranges:
- Year: 1900 to 2021
- Disaster Magnitude Scale: 0 to 4
- Disaster Magnitude Value: -57 to 13,025,874
- Country: Select from the dropdown
- Longitude: 0 to 2439
- Latitude: 0 to 2372

Please enter values within these ranges for accurate predictions.
""")

# Create the main layout
main_layout = pn.Column(
    pn.pane.Markdown("# Disaster Type Prediction"),
    pn.Row(input_panel, output_panel)
)

# Show the app in a new window
main_layout.show()

Model type: <class 'sklearn.ensemble._forest.RandomForestClassifier'>
Scaler type: <class 'sklearn.preprocessing._data.StandardScaler'>
Launching server at http://localhost:52991
